In [149]:
import re
lista_strings = ["Olá, mundo!", "Este é um exemplo de string com @caracteres especiais#"]


def limpar_caracteres_especiais(string):

    return re.sub(r'[^\w\s]', '', string)


lista_limpa = [limpar_caracteres_especiais(string) for string in lista_strings]


for string in lista_limpa:
    print(string)

Olá mundo
Este é um exemplo de string com caracteres especiais


In [150]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [151]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
c:\Users\guido\OneDrive\Documentos\GitHub\Cdados


In [152]:
train = pd.read_excel('Dados\dados_treino.xlsx')
train.head(5)

,Target,Mensagem
0,Positivo,"Leitura rápida, com algumas dicas para expandi..."
1,Neutro,"Coaching virou um grande modismo, assim como c..."
2,Negativo,O fundo da história é pra se pensar. Mas é mui...
3,Neutro,A tradução é péssima e diferente do original. ...
4,Negativo,Achei sem graça esse livro. Tópicos de livros ...


In [153]:
test = pd.read_excel('dados/dados_teste.xlsx')
test.head(5)

,Target,Mensagem
0,Neutro,"É incrível como os sebos não tem noção, vendem..."
1,Negativo,Livro sem nenhuma coisa de diferente. Autor ba...
2,Negativo,Bem fraco. Achei muito forçada a ligação com o...
3,Neutro,A caixa veio rasgada. como fazia mais de sete ...
4,Negativo,"Que livro chato, gente quase não consegui term..."


In [154]:
df = pd.read_excel('dados/dados_treino.xlsx')

coluna_especifica = df['Mensagem']



def limpar_caracteres_especiais(string):

    return re.sub(r'[^\w\s]', '', string)


nova_coluna_especifica = [limpar_caracteres_especiais(string) for string in coluna_especifica]
nova_coluna_especifica=[string.lower() for string in nova_coluna_especifica]
train.Mensagem=nova_coluna_especifica
train


,Target,Mensagem
0,Positivo,leitura rápida com algumas dicas para expandir...
1,Neutro,coaching virou um grande modismo assim como cr...
2,Negativo,o fundo da história é pra se pensar mas é muit...
3,Neutro,a tradução é péssima e diferente do original a...
4,Negativo,achei sem graça esse livro tópicos de livros a...
...,...,...
745,Negativo,conteúdo superficial onde se propõe nas dicas ...
746,Negativo,péssimo
747,Negativo,autor confunde tecnicas de coaching com religi...
748,Negativo,livro superficial escrito para impressionar os...


In [175]:
def separa_palavra(lista):
    palavras=[]
    for frase in lista:
        palavra=frase.split()
        for e in palavra:
            palavras.append(e)
    palavras_series=pd.Series(palavras)
    tabela_palavras=palavras_series.value_counts()
    return tabela_palavras


In [176]:
def separa_palavra_relativa(lista):
    palavras=[]
    for frase in lista:
        palavra=frase.split()
        for e in palavra:
            palavras.append(e)
    palavras_series=pd.Series(palavras)
    tabela_palavras=palavras_series.value_counts(True)
    return tabela_palavras

In [177]:
train.Target=train.Target.astype('category')
test.Target=test.Target.astype('category')
train_positivo = train.loc[train.Target=='Positivo' , :]
train_negativo = train.loc[train.Target=='Negativo' , :]
train_neutro = train.loc[train.Target=='Neutro' , :]
tabela_palavras=separa_palavra(train.Mensagem)
tabela_train_positivo=separa_palavra(train_positivo.Mensagem)
tabela_train_negativo=separa_palavra(train_negativo.Mensagem)
tabela_train_neutro=separa_palavra(train_neutro.Mensagem)

In [178]:
tabela_palavras_relativa=separa_palavra_relativa(train.Mensagem)
tabela_train_positivo_relativa=separa_palavra_relativa(train_positivo.Mensagem)
tabela_train_negativo_relativa=separa_palavra_relativa(train_negativo.Mensagem)
tabela_train_neutro_relativa=separa_palavra_relativa(train_neutro.Mensagem)

In [181]:
def probabilidade_frase(frase):
    tabela_train_positivo_relativa=separa_palavra_relativa(train_positivo.Mensagem)
    tabela_train_negativo_relativa=separa_palavra_relativa(train_negativo.Mensagem)
    tabela_train_neutro_relativa=separa_palavra_relativa(train_neutro.Mensagem)
    Positivo=1
    Negativo=1
    Neutro=1
    frase_split=frase.split()
    for i in frase_split:
        chance_pos=tabela_train_positivo_relativa[i]
        prob_pos=Positivo*chance_pos
    
        chance_neg=tabela_train_negativo_relativa[i]
        prob_neg=Negativo*chance_neg

        chance_neu=tabela_train_neutro_relativa[i]
        prob_neu=Neutro*chance_neg
    if prob_neg>prob_pos and prob_neg>prob_neu:
        return ('Negativo')
    elif prob_pos>prob_neg and prob_pos>prob_neu:
        return ('Positivo')
    else:
        return('Neutro')

print(probabilidade_frase('me mostrou a não confiar mais nos best sellers'))

KeyError: 'mostrou'